In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
def load_csvs(folder):
    data_path = f"C:\\Users\\roybo\\OneDrive - University College London\\PhD\\PhD_Prog\\017_VQ_sCE\\Phase3\\{folder}"
    focal_L1 = pd.read_csv(f"{data_path}\\contrast_test_focal_L1.csv", index_col=0, header=[0, 1])
    global_L1 = pd.read_csv(f"{data_path}\\contrast_test_global.csv", index_col=0, header=[0, 1])
    textures = pd.read_csv(f"{data_path}\\contrast_test_texture.csv", index_col=0)

    def _change_expt_name(expt):
        expt = folder.replace("_nores", "") + '_' +  expt
        return expt

    focal_L1.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in focal_L1.columns])
    global_L1.columns = pd.MultiIndex.from_tuples([(c[0], _change_expt_name(c[1])) for c in global_L1.columns])
    textures.columns = [_change_expt_name(c) for c in textures.columns]

    return focal_L1, global_L1, textures

In [3]:
def merge_kidney_rois(df):
    right_kidney = df["RK"]
    no_left_kidney = df["LK"].isna().any(axis=1)
    right_kidney.loc[~no_left_kidney] = (right_kidney.loc[~no_left_kidney] + df.loc[~no_left_kidney, "LK"]) / 2
    right_kidney.columns = pd.MultiIndex.from_product([["KD"], right_kidney.columns])
    df = pd.concat([df, right_kidney], axis=1)
    df.drop(["RK", "LK", "VC"], axis=1, inplace=True)
    return df

In [46]:
focal_deep, global_deep, texture_deep = load_csvs("deep")
focal_deep = merge_kidney_rois(focal_deep)

focal_shallow, global_shallow, texture_shallow = load_csvs("shallow")
focal_shallow = merge_kidney_rois(focal_shallow)

focal_multi, global_multi, texture_multi = load_csvs("multi")
focal_multi = merge_kidney_rois(focal_multi)

C:\Users\roybo\AppData\Local\Temp\ipykernel_1096\2063971159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_kidney.loc[~no_left_kidney] = (right_kidney.loc[~no_left_kidney] + df.loc[~no_left_kidney, "LK"]) / 2
C:\Users\roybo\AppData\Local\Temp\ipykernel_1096\2063971159.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(["RK", "LK", "VC"], axis=1, inplace=True)
C:\Users\roybo\AppData\Local\Temp\ipykernel_1096\2063971159.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  right_kidney.loc[~no_left_kidney] = (right_kidney.loc[~no_left_kid

In [25]:
global_df = pd.concat([global_deep, global_shallow, global_multi], axis=1)
focal_df = pd.concat([focal_deep, focal_shallow, focal_multi], axis=1)
texture_df = pd.concat([texture_deep, texture_shallow, texture_multi], axis=1)
texture_df.drop(labels=["shallow_NCE", "shallow_CE", "multi_NCE", "multi_CE"], axis=1, inplace=True)
texture_df.rename({"deep_NCE": "NCE", "deep_CE": "CE"}, axis=1, inplace=True)

In [37]:
# expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]
expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
# expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

global_df.loc[:, pd.IndexSlice[["MSE", "pSNR", "SSIM"], expts]].quantile([0.05, 0.5, 0.95])
# focal_df.loc[:, pd.IndexSlice["AO", expts]].quantile([0.05, 0.5, 0.95])
# texture_df.loc[:, ["NCE", "CE"] + expts].quantile([0.05, 0.5, 0.95])

MSE                                                    pSNR  \
        deep_ce_vq0   deep_ce_vq1  deep_jo_vq512   deep_jo_vq64 deep_ce_vq0   
0.05    2604.169414    2240.17262    2358.254596    2263.947472   15.710839   
0.50   48792.749900   48853.26030   48966.646180   48866.704006   22.658872   
0.95  243798.090050  244577.92886  242992.453420  242752.413093   35.388915   

                                                   SSIM              \
     deep_ce_vq1 deep_jo_vq512 deep_jo_vq64 deep_ce_vq0 deep_ce_vq1   
0.05   15.697673     15.725340    15.729120    0.790924    0.793489   
0.50   22.653490     22.643421    22.652295    0.856741    0.870619   
0.95   36.049544     35.817912    36.001917    0.915767    0.924727   

                                 
     deep_jo_vq512 deep_jo_vq64  
0.05      0.788741     0.795082  
0.50      0.870138     0.868356  
0.95      0.924055     0.924589

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(global_df["MSE"])
plt.xticks(ticks=np.arange(1, global_df["MSE"].shape[1] + 1), labels=global_df["MSE"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(global_df["pSNR"])
plt.xticks(ticks=np.arange(1, global_df["pSNR"].shape[1] + 1), labels=global_df["pSNR"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(global_df["SSIM"])
plt.xticks(ticks=np.arange(1, global_df["SSIM"].shape[1] + 1), labels=global_df["SSIM"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [33]:
print(f"Alpha = {0.05 / 6}")
# expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]
# expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

print(f"MSE: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['MSE', expts]].values.T.tolist())}")
print(f"pSNR: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['pSNR', expts]].values.T.tolist())}")
print(f"SSIM: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['SSIM', expts]].values.T.tolist())}")

Alpha = 0.008333333333333333
MSE: KruskalResult(statistic=0.056283919751194844, pvalue=0.9965080021064306)
pSNR: KruskalResult(statistic=0.056283919751194844, pvalue=0.9965080021064306)
SSIM: KruskalResult(statistic=2.989267638270576, pvalue=0.39328285694273846)


In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(3, 1, 1)
plt.boxplot(global_df["L1"])
plt.xticks(ticks=np.arange(1, global_df["L1"].shape[1] + 1), labels=global_df["L1"].columns, rotation="vertical")
plt.subplot(3, 1, 2)
plt.boxplot(focal_df["AO"])
plt.xticks(ticks=np.arange(1, focal_df["AO"].shape[1] + 1), labels=focal_df["AO"].columns, rotation="vertical")
plt.subplot(3, 1, 3)
plt.boxplot(focal_df["KD"])
plt.xticks(ticks=np.arange(1, focal_df["KD"].shape[1] + 1), labels=focal_df["KD"].columns, rotation="vertical")
plt.tight_layout()
plt.show()

In [41]:
print("Baselines")
print(f"Alpha = {0.05 / 3}")
expts = ["deep_ce_vq0", "shallow_ce_vq0", "multi_ce_vq0"]

print(f"Global L1: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")
print(f"Focal AO deep vs shallow: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'deep_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'shallow_ce_vq0']])}")
print(f"Focal AO deep vs multi: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'deep_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']])}")
print(f"Focal AO shallow vs multi: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'shallow_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']])}")

Baselines
Alpha = 0.016666666666666666
Global L1: KruskalResult(statistic=0.1570618417499361, pvalue=0.9244734739609042)
Focal AO: KruskalResult(statistic=7.180556305698076, pvalue=0.027590654936542107)
Focal KD: KruskalResult(statistic=2.5607345395625316, pvalue=0.27793520450419906)
Texture: KruskalResult(statistic=1.528490413178531, pvalue=0.46568529454597685)
Focal AO deep vs shallow: MannwhitneyuResult(statistic=367.0, pvalue=0.025035377189050925)
Focal AO deep vs multi: MannwhitneyuResult(statistic=366.0, pvalue=0.0264943971280104)
Focal AO shallow vs multi: MannwhitneyuResult(statistic=220.0, pvalue=0.33372205862936943)


In [47]:
print(f"Alpha = {0.05 / 12}")
# expts = ["deep_ce_vq0", "deep_ce_vq1", "deep_jo_vq512", "deep_jo_vq64"]
# expts = ["shallow_ce_vq0", "shallow_ce_vq512", "shallow_jo_vq256", "shallow_jo_vq8"]
expts = ["multi_ce_vq0", "multi_ce_vq128-128-256-512-512", "multi_jo_vq128-128-512-512-1024", "multi_jo_vq128-64-256-32-512"]

print(f"Global L1: {stats.kruskal(*global_df.loc[:, pd.IndexSlice['L1', expts]].values.T.tolist())}")
print(f"Focal AO: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['AO', expts]].values.T.tolist())}")
print(f"Focal KD: {stats.kruskal(*focal_df.loc[:, pd.IndexSlice['KD', expts]].values.T.tolist())}")
print(f"Texture: {stats.kruskal(*texture_df.loc[:, expts].values.T.tolist())}")

print("Multi")
print(f"Focal AO base vs vq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']])}")
print(f"Focal AO base vs jvq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"Focal AO base vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq0']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")
print(f"Focal AO vq-ce vs jvq-ce: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']])}")
print(f"Focal AO vq-ce vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_ce_vq128-128-256-512-512']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")
print(f"Focal AO jvq-ce vs jvq-sr: {stats.mannwhitneyu(focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-128-512-512-1024']], focal_df.loc[:, pd.IndexSlice['AO', 'multi_jo_vq128-64-256-32-512']])}")

print("Texture")
print(f"Texture base vs vq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'])}")
print(f"Texture base vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture base vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq0'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture vq-ce vs jvq-ce: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'])}")
print(f"Texture vq-ce vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_ce_vq128-128-256-512-512'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")
print(f"Texture jvq-ce vs jvq-sr: {stats.mannwhitneyu(texture_df.loc[:, 'multi_jo_vq128-128-512-512-1024'], texture_df.loc[:, 'multi_jo_vq128-64-256-32-512'])}")

Alpha = 0.004166666666666667
Global L1: KruskalResult(statistic=0.3656930300627437, pvalue=0.9472346631194354)
Focal AO: KruskalResult(statistic=26.78913348374897, pvalue=6.518102159091851e-06)
Focal KD: KruskalResult(statistic=6.382645283248962, pvalue=0.09440737223757584)
Texture: KruskalResult(statistic=19.225379596316827, pvalue=0.00024557467948000875)
Multi
Focal AO base vs vq-ce: MannwhitneyuResult(statistic=271.0, pvalue=0.8951305559105456)
Focal AO base vs jvq-ce: MannwhitneyuResult(statistic=202.0, pvalue=0.1731697381246956)
Focal AO base vs jvq-sr: MannwhitneyuResult(statistic=65.0, pvalue=1.2318860354108018e-05)
Focal AO vq-ce vs jvq-ce: MannwhitneyuResult(statistic=197.0, pvalue=0.14103840489970726)
Focal AO vq-ce vs jvq-sr: MannwhitneyuResult(statistic=69.0, pvalue=1.835709957208419e-05)
Focal AO jvq-ce vs jvq-sr: MannwhitneyuResult(statistic=111.0, pvalue=0.00077580834926157)
Texture
Texture base vs vq-ce: MannwhitneyuResult(statistic=195.0, pvalue=0.12955135290440883)
Te